In [ ]:
import numpy as np
import unyt
import swiftsimio as sw
from velociraptor import load
import swiftgalaxy as sg
import functions as fn
from swiftgalaxy import SWIFTGalaxy, MaskCollection
import h5py
dir="/mnt/su3-pro/flamingo/L0200N0360/"

data_h=load(dir+"VR/halos_0008.properties.0")
radius=data_h.radii.rvir

is_mainhalo=data_h.centrals
halo_id=data_h.ids.id-1
mass=data_h.masses.mass_tot
xc=data_h.positions.xcmbp
yc=data_h.positions.ycmbp
zc=data_h.positions.zcmbp
data_h=[]
mainhalo_id=halo_id[is_mainhalo*(mass>10000)]
mainhalo_id=np.array([100,101,102])
mainhalo_id=mainhalo_id.astype(int)
print(halo_id)

[0.0000e+00 1.0000e+00 2.0000e+00 ... 9.9525e+04 9.9526e+04 9.9527e+04] dimensionless


In [22]:
#S>0.2:24.,  25.,  28.,  29.,  32., 101., 102.
#S<0:8., 56., 75., 95., 99
#Offset<0.01:11., 16., 34., 50
#Offset>0.06:72.,  73.,  87., 105
#CoM different>0.15: 1.,   2.,  10.,  21.,  57.,  73.,  85., 105
mainhalo_id=np.array([749])
for id in mainhalo_id:
 
  centre=np.array([xc[id],yc[id],zc[id]])*unyt.Mpc
  r=radius[id]*1.5
 
  sgi=sg.SWIFTGalaxy("/../../mnt/su3-pro/flamingo/L0200N0720/snapshots/flamingo_0008/flamingo_0008.hdf5",
                   sg.Standalone(centre=centre,velocity_centre=np.array([0,0,0])*
                                 unyt.km/unyt.s,spatial_offsets=[[-r,r],[-r,r],[-r,r]],extra_mask=None))
  print(len(sgi.gas.spherical_coordinates.r))
  mask=sg.MaskCollection(dark_matter=(sgi.dark_matter.spherical_coordinates.r<r),
                       gas=(sgi.gas.spherical_coordinates.r<r)*(sgi.gas.temperatures>00000),
                       
                       stars=sgi.stars.spherical_coordinates.r<r)
  #
  sgi.mask_particles(mask)
  print(len(sgi.gas.spherical_coordinates.r))
  x_dm=np.array(sgi.dark_matter.cartesian_coordinates.x)
  y_dm=np.array(sgi.dark_matter.cartesian_coordinates.y)
  z_dm=np.array(sgi.dark_matter.cartesian_coordinates.z)
  x_g=np.array(sgi.gas.cartesian_coordinates.x)
  y_g=np.array(sgi.gas.cartesian_coordinates.y)
  z_g=np.array(sgi.gas.cartesian_coordinates.z)
  PartType1=np.array([x_dm,y_dm,z_dm]).T
  PartType2=np.array([x_g,y_g,z_g]).T
  f = h5py.File('/home/jyang/data/Flamingo/L0200N0360/halo_particles/'+str(int(id))+'around.hdf5', 'w')
  dm = f.create_group("PartType2")
  dm.create_dataset("Coordinates", data=PartType1)
  g= f.create_group("PartType1")
  g.create_dataset("Coordinates", data=PartType2)
  f.create_dataset("Header", data=[0,0,0,0,0,0,0,0])
  f.close()

/home/jyang/anaconda3/envs/myenv/lib/python3.12/site-packages/swiftsimio/_array_functions.py:438: RuntimeWarning: Mixing arguments with and without cosmo_factors, continuing assuming provided cosmo_factor (a at a=1.0) for all arguments.
  warnings.warn(
/home/jyang/anaconda3/envs/myenv/lib/python3.12/site-packages/swiftgalaxy/reader.py:2087: RuntimeWarning: Translation assumed to be in comoving (not physical) coordinates.
  warn(


249674


/home/jyang/anaconda3/envs/myenv/lib/python3.12/site-packages/swiftsimio/_array_functions.py:665: RuntimeWarning: Mixing arguments with and without cosmo_factors, continuing assuming provided cosmo_factor ((a**2.0)**0.5 at a=1.0) for all arguments.
  warnings.warn(


72785


In [8]:
f = h5py.File('/home/jyang/data/Flamingo/L0200N0360/halo_particles/749.hdf5', 'w')
dm = f.create_group("PartType2")
dm.create_dataset("Coordinates", data=PartType1)
g= f.create_group("PartType1")
g.create_dataset("Coordinates", data=PartType2)
f.create_dataset("Header", data=[0,0,0,0,0,0,0,0])
f.close()